In [ ]:
%pip install langchain==0.1.0 openai==1.7.2 langchain-openai==0.3.8 langchain-community==0.0.12 langchainhub==0.1.14

In [ ]:
%pip install python-dotenv


In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi")
llm.invoke("Tell me a joke")

In [ ]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage

llm = Ollama(model="phi")

messages = [
    SystemMessage(content="You are an AI assistant that tells jokes."),
    HumanMessage(content="Tell me a joke.")
]

response = llm.invoke(messages)
print(response) 


In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate.from_template(
    "Tell me historical fact about the {event} in {location}.",
)

In [ ]:
prompt_template.format(event="World War II", location="Europe")

In [ ]:
import os
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains import RetrievalQA
from langchain.prompts import (
    PromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate
)

In [ ]:
%pip install neo4j

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
import os

# 1. Khởi tạo HuggingFace Embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 2. Kết nối Neo4j và build vectorstore
vectorstore = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="medical_full_index",  # 1 index chung
    node_label="Disease|Symptom|Treatment|Prevention|DiagnoseMethod|Cause",
    text_node_properties=["name", "overview"],
    embedding_node_property="embedding"
)



In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
import os

# 1. Khởi tạo model embedding
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 2. Thông tin kết nối Neo4j
neo4j_config = {
    "url": os.getenv("NEO4J_URL"),
    "username": os.getenv("NEO4J_USERNAME"),
    "password": os.getenv("NEO4J_PASSWORD")  # <-- chỉnh lại nếu cần
}

# 3. Các loại node cần tạo riêng
node_types = ["Disease", "Symptom", "Treatment", "Prevention", "DiagnoseMethod", "Cause"]

# 4. Duyệt từng loại node và tạo vectorstore riêng biệt
vectorstores = {}
for node_label in node_types:
    index_name = f"index_{node_label.lower()}"
    properties = ["name", "overview"] if node_label == "Disease" else ["name"]

    print(f"👉 Đang tạo vectorstore cho {node_label}...")

    vectorstores[node_label] = Neo4jVector.from_existing_graph(
        embedding=embedding_model,
        index_name=index_name,
        node_label=node_label,
        text_node_properties=properties,
        embedding_node_property="embedding",
        **neo4j_config
    )

print("✅ Đã tạo vectorstore cho tất cả các node!")


In [ ]:
from neo4j import GraphDatabase
import os

# Kết nối
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URL"), 
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

def drop_all_vector_indexes():
    with driver.session() as session:
        indexes = session.run("""
        SHOW INDEXES YIELD name, type
        WHERE type = "VECTOR"
        RETURN name
        """)
        for record in indexes:
            index_name = record["name"]
            print(f"Dropping index {index_name}...")
            session.run(f"DROP INDEX {index_name} IF EXISTS")

drop_all_vector_indexes()
print("✅ Đã drop toàn bộ vector index!")


### Vector Embedding

In [ ]:
from neo4j_vector import Neo4jVectorManager

# Config kết nối Neo4j
neo4j_config = {
    "url": "bolt://localhost:7687",
    "username": "neo4j",
    "password": "12345678"
}

labels = ["Disease", "Cause", "Symptom", "Treatment", "Prevention", "DiagnoseMethod"]

# 1. Khởi tạo
manager = Neo4jVectorManager(neo4j_config)

# 2. Tạo embeddings + indexes (chỉ cần chạy 1 lần)
manager.process_all_labels(labels)



# 4. Đóng kết nối
manager.close()


🧩 [DiagnoseMethod] Xét nghiệm hormone sinh dục. | Score: 0.9072
🧩 [DiagnoseMethod] Để chẩn đoán bệnh, trước hết có thể thông qua kết quả khám lâm sàng các vết loét, khám hạch bẹn sưng và loại trừ các bệnh lây truyền qua đường tình dục khác. | Score: 0.8994
🧩 [DiagnoseMethod] Dựa vào dấu hiệu lâm sàng của bệnh:  Bệnh nhân chỉ cần có Một yếu tố chính về tình dục và Hai triệu chứng toàn thân (Một chính, Hai phụ) đã nêu trong phần triệu chứng thì được chẩn đoán là mãn dục ở nam giới. | Score: 0.8969


## Find k result

In [4]:
from neo4j_vector import Neo4jVectorManager

# Config kết nối Neo4j
neo4j_config = {
    "url": "bolt://localhost:7687",
    "username": "neo4j",
    "password": "12345678"
}



# 1. Khởi tạo
manager = Neo4jVectorManager(neo4j_config)


# 3. Tìm kiếm semantic similarity
results = manager.similarity_search_git("Disease", "Ấu dâm là gì ?", top_k=1)

for label, text, score in results:
    print(f"[{label}] {text} (Score: {score:.4f})")


# 4. Đóng kết nối
manager.close()


[Disease] Ấu dâm là gì?
Ấu dâm là một chứng rối loạn tình dục gồm những ham muốn tình dục mạnh mẽ, liên tục mà đối tượng muốn quan hệ là trẻ em chưa hoặc mới dậy thì.
Các hành vi ấu dâm bao gồm: nhìn, vuốt ve, thủ dâm và ép quan hệ tình dục với trẻ em. Có một số trường hợp đồng tính nam thích quan hệ với các cậu bé, hầu hết người có xu hướng ấu dâm là nam giới, nhưng cũng có những trường hợp người bệnh là nữ giới.
Dưới ảnh hưởng của giới truyền thông, ấu dâm được hiểu là các hành động lạm dụng tình dục với trẻ em. Tuy nhiên, định nghĩa này đã phản ánh sai tình hình chung của những người mắc bệnh ấu dâm, làm cho căn bệnh này càng khó nghiên cứu và thu thập số liệu.
Ấu dâm là như thế nào?
Cần khẳng định rằng ấu dâm là bệnh, không phải là tội. Không phải bất kỳ ai mắc bệnh ấu dâm cũng có hành vi tình dục với trẻ em, cũng như những người có hành vi tình dục với trẻ em cũng không chắc chắn là có bệnh ấu dâm. (Score: 0.9137)


In [4]:
from neo4j_vector import Neo4jVectorManager

# Config kết nối Neo4j
neo4j_config = {
    "url": "bolt://localhost:7687",
    "username": "neo4j",
    "password": "12345678"
}

# 1. Khởi tạo
manager = Neo4jVectorManager(neo4j_config)

query = "Nguyên nhân ấu dâm ?"

label = manager.detect_label_from_query_git(query)
print(f"🔎 Label phát hiện: {label}")

results = manager.similarity_search_git(label=label, query_text=query)

for lbl, text, score in results:
    print(f"[{lbl}] {text} (Score: {score:.4f})")

manager.close()


🔎 Label phát hiện: Cause
[Cause] Các yếu tố được nghiên cứu và giả thuyết đưa ra nguyên nhân gây bệnh ấu dâm như: Chỉ số IQ và trí nhớ kém; Ít chất trắng trong não bộ; Ít hormone testosterone; Các vấn đề trong não bộ. (Score: 0.8453)
[Cause] Hiện nay các nhà khoa học vẫn chưa biết chính xác nguyên nhân gây ra ấu dâm là gì. Do tính cách và hoàn cảnh mỗi người đều khác nhau nên rất khó để có thể nghiên cứu nguyên nhân từ khía cạnh tâm lý. (Score: 0.8438)
[Cause] Có một số chuyên gia cho rằng tình trạng này có liên quan đến các nhân tố tâm lý xã hội chứ không phải các nhân tố sinh học, lại có một số bác sĩ cho rằng nhân tố tính cách có ảnh hưởng đến bệnh nhân như: các vấn đề về sự gắn bó hoặc phụ thuộc vào gia cảnh bất thường, bị quấy rối khi còn nhỏ cũng có thể là một trong những nguyên nhân gây ra ấu dâm. (Score: 0.8305)
[Cause] Rối loạn tình dục. (Score: 0.8203)
[Cause] Các bệnh lây truyền qua đường tình dục; (Score: 0.8069)


In [ ]:
import gradio as gr
from neo4j_vector import Neo4jVectorManager


# Cấu hình kết nối
neo4j_config = {
    "url": "bolt://localhost:7687",
    "username": "neo4j",
    "password": "12345678"
}

def search_similar_nodes(query):
    manager = Neo4jVectorManager(neo4j_config)
    try:
        label = manager.detect_label_from_query_git(query)
        results = manager.similarity_search_git(label=label, query_text=query)

        output_lines = [f"🔎 Đã phát hiện label: {label}\n"]
        for lbl, text, score in results:
            output_lines.append(f"🧩 {text}")
        return "\n\n".join(output_lines)
    except Exception as e:
        return f"❌ Lỗi: {str(e)}"
    finally:
        manager.close()

# Giao diện Gradio
demo = gr.Interface(
    fn=search_similar_nodes,
    inputs=gr.Textbox(lines=2, placeholder="Nhập câu hỏi như: 'AIDS là gì?'"),
    outputs=gr.Textbox(label="Kết quả tương tự"),
    title="🧠 Semantic Search (Neo4j + SimCSE)",
    description="Tìm kiếm các node y học tương tự trong đồ thị Neo4j theo vector embedding (PhoBERT + cosine)."
)

demo.launch(share=False)


e:\Master\Data & Structure\Paper_KG\Doan\graphrag-chatbot-medical\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------


In [5]:
import gradio as gr
from neo4j_vector import Neo4jVectorManager
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Cấu hình kết nối Neo4j
neo4j_config = {
    "url": "bolt://localhost:7687",
    "username": "neo4j",
    "password": "12345678"
}

# Prompt tóm tắt kết quả bằng LLM
template = """
Bạn là một trợ lý y tế thông minh. Dưới đây là các đoạn thông tin gần giống với câu hỏi:

Câu hỏi: {question}

Các đoạn liên quan:
{contexts}

Hãy tóm tắt lại một câu trả lời ngắn gọn, đúng chuyên môn.
"""

prompt = PromptTemplate.from_template(template)
llm = Ollama(model="phi")  # Bạn có thể đổi thành "gemma", "mistral", hoặc OpenAI
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Hàm xử lý tìm kiếm + tóm tắt
def search_and_summarize(query):
    manager = Neo4jVectorManager(neo4j_config)
    try:
        label = manager.detect_label_from_query_git(query)
        results = manager.similarity_search_git(label=label, query_text=query, top_k=5)

        if not results:
            return f"Không tìm thấy kết quả cho label {label}."

        # Format cho hiển thị
        display_output = f"🔎 Đã phát hiện label: {label}\n\n"
        for i, (_, name, _) in enumerate(results, 1):
            display_output += f"🧩 {i}. {name}\n"

        # Tóm tắt bằng LLM
        contexts = "\n".join(f"- {name}" for _, name, _ in results)
        answer = llm_chain.run({"question": query, "contexts": contexts})

        return f"{display_output}\n\n🧠 **Câu trả lời bởi LLM:**\n{answer}"

    except Exception as e:
        return f"❌ Lỗi: {str(e)}"
    finally:
        manager.close()

# Giao diện Gradio
demo = gr.Interface(
    fn=search_and_summarize,
    inputs=gr.Textbox(lines=2, placeholder="Nhập câu hỏi như: 'Nguyên nhân bệnh tiểu đường?'"),
    outputs=gr.Textbox(label="Kết quả + Trả lời bởi LLM", lines=10),
    title="🧠 Semantic QA (Neo4j + SimCSE + LLM)",
    description="Tìm kiếm tri thức y học từ Neo4j và trả lời tự nhiên bằng LLM (phi/mistral/gemma)."
)

demo.launch(share=False)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------


e:\Master\Data & Structure\Paper_KG\Doan\graphrag-chatbot-medical\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
from langchain.chains import RetrievalQA



# Gộp nhiều retrievers
retrievers = [
    store.as_retriever(search_kwargs={"k": 10}) for store in vectorstores.values()
]

# Gộp thành 1 retriever duy nhất
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=retrievers,
    weights=[1] * len(retrievers)  # mỗi vectorstore trọng số bằng nhau
)

# Tạo chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm = Ollama(model="phi"),  # hoặc Huggingface
    retriever=ensemble_retriever,
    chain_type="stuff"
)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import EnsembleRetriever
from langchain.llms import Ollama

# Tạo retrievers với search_kwargs riêng
retrievers = [
    vectorstores["Disease"].as_retriever(search_kwargs={"k": 3}),
    vectorstores["Cause"].as_retriever(search_kwargs={"k": 3}),
    vectorstores["Symptom"].as_retriever(search_kwargs={"k": 2}),
    vectorstores["Treatment"].as_retriever(search_kwargs={"k": 2}),
    vectorstores["Prevention"].as_retriever(search_kwargs={"k": 1}),
    vectorstores["DiagnoseMethod"].as_retriever(search_kwargs={"k": 1}),
]

# Trọng số ưu tiên (Disease & Cause nặng hơn)
weights = [3, 3, 2, 2, 1, 1]

# Gộp retrievers
ensemble_retriever = EnsembleRetriever(
    retrievers=retrievers,
    weights=weights
)

# Tạo QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=Ollama(model="phi"),
    retriever=ensemble_retriever,
    chain_type="stuff"  # Có thể thay bằng map_reduce, refine nếu cần
)


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-embedding" # hoặc "cpu"
)

vector = embeddings.embed_query("Test embedding")
print(vector[:10])  # In thử 10 phần tử đầu


In [ ]:
from langchain_community.vectorstores import Neo4jVector

vector_index = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    search_type="hybrid",  # or "similarity"
    node_label="Disease",  # 👈 lấy tất cả
    text_node_properties=["name", "overview"],  # đảm bảo các node có trường name
    embedding_node_property="embedding"
)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # ví dụ: Ollama(model="phi")
    retriever=vector_index.as_retriever(search_kwargs={"k": 5})
)


In [ ]:
query = "Nguyên nhân của AIDS là gì?"
response = qa_chain.run(query)
print("🤖 Trả lời:\n", response)


In [ ]:
%pip install gradio

In [ ]:
import gradio as gr

def ask_graph_rag(question):
    return qa_chain.run(question)

gr.Interface(
    fn=ask_graph_rag,
    inputs="text",
    outputs="text",
    title="GraphRAG Medical Chatbot 🧠",
    description="Truy vấn tri thức từ đồ thị y tế ",
).launch(share=True)
